In [4]:
pip install yahoo_fin

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [6]:
from yahoo_fin import stock_info as si

In [10]:
si.tickers_nasdaq()

['AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AADI',
 'AADR',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPL',
 'AATC',
 'AAWW',
 'AAXJ',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABGI',
 'ABIO',
 'ABMD',
 'ABNB',
 'ABOS',
 'ABSI',
 'ABST',
 'ABTX',
 'ABUS',
 'ABVC',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACB',
 'ACBA',
 'ACBAU',
 'ACBAW',
 'ACBI',
 'ACCD',
 'ACER',
 'ACET',
 'ACEV',
 'ACEVU',
 'ACEVW',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACHC',
 'ACHL',
 'ACHV',
 'ACIU',
 'ACIW',
 'ACKIT',
 'ACKIU',
 'ACKIW',
 'ACLS',
 'ACMR',
 'ACNB',
 'ACOR',
 'ACQR',
 'ACQRU',
 'ACQRW',
 'ACRS',
 'ACRX',
 'ACST',
 'ACT',
 'ACTD',
 'ACTDU',
 'ACTDW',
 'ACTG',
 'ACVA',
 'ACWI',
 'ACWX',
 'ACXP',
 'ADAG',
 'ADALU',
 'ADAP',
 'ADBE',
 'ADER',
 'ADERU',
 'ADERW',
 'ADES',
 'ADGI',
 'ADI',
 'ADIL',
 'ADILW',
 'ADMA',
 'ADMP',
 'ADN',
 'ADNWW',
 'ADOC',
 'ADOCR',
 'ADOCW',
 'ADP',
 'ADPT',
 'ADRE',
 'ADSK',
 'ADTN',
 'ADTX',
 'ADUS',
 'ADV',
 'ADVM',
 'ADVWW',
 'ADXN',
 'ADXS',
 'AEAC',
 'AEACU',
 'AEACW',
 'AEAEU',
 'AEHA'

In [11]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [152]:
from sklearn import preprocessing, utils 
import numpy as np
from collections import deque 
import pandas as pd

In [17]:
features = ['adjclose', 'volume', 'open', 'high', 'low']

In [146]:
# loads data from yahoo finance based on a ticker string, ensures all requist columns are in the 
# DataFrame, adds date col if it doesn't exist (uses index values), splits the data by date into 
# training and testing sets 
def get_data(ticker, window=50, predict=1):
    try:
        df = si.get_data(ticker)
    except TypeError:
        print('ticker not string type')
    
    output = {}
    output['DataFrame'] = df.copy()

    for col in features:
        if col not in df.columns:
            return print(f'{col} missing')

    if 'date' not in df.columns:
        df['date'] = df.index

    # for col in features:
    #     scaler = preprocessing.MinMaxScaler()
    #     df[col] = scaler.fit_transform()
    # scaling data unfinished 

    df['future'] = df['adjclose'].shift(-predict)
    window_seq = np.array(df[features].tail(window))
    df.dropna(inplace=True)

    feat_date = deque(maxlen=window)
    feat_date_future = []
    temp = zip(df[features + ['date']].values, df['future'].values)

    for x, y in temp:
        feat_date.append(x)
        if len(feat_date) == window:
            feat_date_future.append([np.array(feat_date), y])

    window_seq = list([i[:len(features)] for i in feat_date]) + list(window_seq)
    window_seq = np.array(window_seq).astype(np.float32)
    
    output['window_seq'] = window_seq

    x = []
    y = []

    for X, Y in feat_date_future:
        x.append(X)
        y.append(Y)

    x = np.array(x)
    y = np.array(y)

    train_size = int(.8 * len(x))
    output['train_x'] = x[:train_size]
    output['train_y'] = y[:train_size]
    output['test_x'] = x[train_size:]
    output['test_y'] = y[train_size:]

    utils.shuffle(output['train_x'], output['train_y'])
    utils.shuffle(output['test_x'], output['test_y'])

    dates = output['test_x'][:, -1, -1]
    output['DataFrame_test'] = output['DataFrame'].loc[dates]
    # output['DataFrame_test'].drop_duplicates(keep='first')
    output['DataFrame_test'] = output['DataFrame_test'][~output['DataFrame_test'].index.duplicated(keep='first')]
    output['train_x'] = output['train_x'][:, :, :len(features)].astype(np.float32)
    output['test_x'] = output['test_x'][:, :, :len(features)].astype(np.float32)
    return output

In [147]:
get_data('AAPL')

{'DataFrame':                   open        high         low       close    adjclose  \
 1980-12-12    0.128348    0.128906    0.128348    0.128348    0.100453   
 1980-12-15    0.122210    0.122210    0.121652    0.121652    0.095213   
 1980-12-16    0.113281    0.113281    0.112723    0.112723    0.088224   
 1980-12-17    0.115513    0.116071    0.115513    0.115513    0.090408   
 1980-12-18    0.118862    0.119420    0.118862    0.118862    0.093029   
 ...                ...         ...         ...         ...         ...   
 2021-11-19  157.649994  161.020004  156.529999  160.550003  160.550003   
 2021-11-22  161.679993  165.699997  161.000000  161.020004  161.020004   
 2021-11-23  161.119995  161.800003  159.059998  161.410004  161.410004   
 2021-11-24  160.750000  162.139999  159.639999  161.940002  161.940002   
 2021-11-26  159.570007  160.449997  156.360001  156.809998  156.809998   
 
                volume ticker  
 1980-12-12  469033600   AAPL  
 1980-12-15  17588480

In [150]:
def shuffle_in_unison(a, b):
    # shuffle two arrays in the same way
    state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(state)
    np.random.shuffle(b)

def load_data(ticker, n_steps=50, scale=True, shuffle=True, lookup_step=1, split_by_date=True,
                test_size=0.2, feature_columns=['adjclose', 'volume', 'open', 'high', 'low']):
    """
    Loads data from Yahoo Finance source, as well as scaling, shuffling, normalizing and splitting.
    Params:
        ticker (str/pd.DataFrame): the ticker you want to load, examples include AAPL, TESL, etc.
        n_steps (int): the historical sequence length (i.e window size) used to predict, default is 50
        scale (bool): whether to scale prices from 0 to 1, default is True
        shuffle (bool): whether to shuffle the dataset (both training & testing), default is True
        lookup_step (int): the future lookup step to predict, default is 1 (e.g next day)
        split_by_date (bool): whether we split the dataset into training/testing by date, setting it 
            to False will split datasets in a random way
        test_size (float): ratio for test data, default is 0.2 (20% testing data)
        feature_columns (list): the list of features to use to feed into the model, default is everything grabbed from yahoo_fin
    """
    # see if ticker is already a loaded stock from yahoo finance
    if isinstance(ticker, str):
        # load it from yahoo_fin library
        df = si.get_data(ticker)
    elif isinstance(ticker, pd.DataFrame):
        # already loaded, use it directly
        df = ticker
    else:
        raise TypeError("ticker can be either a str or a `pd.DataFrame` instances")
    # this will contain all the elements we want to return from this function
    result = {}
    # we will also return the original dataframe itself
    result['df'] = df.copy()
    # make sure that the passed feature_columns exist in the dataframe
    for col in feature_columns:
        assert col in df.columns, f"'{col}' does not exist in the dataframe."
    # add date as a column
    if "date" not in df.columns:
        df["date"] = df.index
    if scale:
        column_scaler = {}
        # scale the data (prices) from 0 to 1
        for column in feature_columns:
            scaler = preprocessing.MinMaxScaler()
            df[column] = scaler.fit_transform(np.expand_dims(df[column].values, axis=1))
            column_scaler[column] = scaler
        # add the MinMaxScaler instances to the result returned
        result["column_scaler"] = column_scaler
    # add the target column (label) by shifting by `lookup_step`
    df['future'] = df['adjclose'].shift(-lookup_step)
    # last `lookup_step` columns contains NaN in future column
    # get them before droping NaNs
    last_sequence = np.array(df[feature_columns].tail(lookup_step))
    # drop NaNs
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=n_steps)
    for entry, target in zip(df[feature_columns + ["date"]].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == n_steps:
            sequence_data.append([np.array(sequences), target])
    # get the last sequence by appending the last `n_step` sequence with `lookup_step` sequence
    # for instance, if n_steps=50 and lookup_step=10, last_sequence should be of 60 (that is 50+10) length
    # this last_sequence will be used to predict future stock prices that are not available in the dataset
    last_sequence = list([s[:len(feature_columns)] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)
    # add to result
    result['last_sequence'] = last_sequence
    # construct the X's and y's
    X, y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        y.append(target)
    # convert to numpy arrays
    X = np.array(X)
    y = np.array(y)
    if split_by_date:
        # split the dataset into training & testing sets by date (not randomly splitting)
        train_samples = int((1 - test_size) * len(X))
        result["X_train"] = X[:train_samples]
        result["y_train"] = y[:train_samples]
        result["X_test"]  = X[train_samples:]
        result["y_test"]  = y[train_samples:]
        if shuffle:
            # shuffle the datasets for training (if shuffle parameter is set)
            shuffle_in_unison(result["X_train"], result["y_train"])
            shuffle_in_unison(result["X_test"], result["y_test"])
    else:    
        # split the dataset randomly
        result["X_train"], result["X_test"], result["y_train"], result["y_test"] = train_test_split(X, y, 
                                                                                test_size=test_size, shuffle=shuffle)
    # get the list of test set dates
    dates = result["X_test"][:, -1, -1]
    # retrieve test features from the original dataframe
    result["test_df"] = result["df"].loc[dates]
    # remove duplicated dates in the testing dataframe
    result["test_df"] = result["test_df"][~result["test_df"].index.duplicated(keep='first')]
    # remove dates from the training/testing sets & convert to float32
    result["X_train"] = result["X_train"][:, :, :len(feature_columns)].astype(np.float32)
    result["X_test"] = result["X_test"][:, :, :len(feature_columns)].astype(np.float32)
    return result

In [151]:
load_data('AAPL')

{'df':                   open        high         low       close    adjclose  \
 1980-12-12    0.128348    0.128906    0.128348    0.128348    0.100453   
 1980-12-15    0.122210    0.122210    0.121652    0.121652    0.095213   
 1980-12-16    0.113281    0.113281    0.112723    0.112723    0.088224   
 1980-12-17    0.115513    0.116071    0.115513    0.115513    0.090408   
 1980-12-18    0.118862    0.119420    0.118862    0.118862    0.093029   
 ...                ...         ...         ...         ...         ...   
 2021-11-19  157.649994  161.020004  156.529999  160.550003  160.550003   
 2021-11-22  161.679993  165.699997  161.000000  161.020004  161.020004   
 2021-11-23  161.119995  161.800003  159.059998  161.410004  161.410004   
 2021-11-24  160.750000  162.139999  159.639999  161.940002  161.940002   
 2021-11-26  159.570007  160.449997  156.360001  156.809998  156.809998   
 
                volume ticker  
 1980-12-12  469033600   AAPL  
 1980-12-15  175884800   AAP